In [1]:
import os, pandas as pd
import numpy as np
from IPython.display import display, display_html, HTML
from matplotlib import pyplot as plt

pd.set_option('display.max_rows', None)
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [2]:
def gather_f1(dirname, new):
    dirs = [d for d in next(os.walk('./' + dirname))[1] if not d.startswith('.')]
    # print(dirs)
    results = []
    years = ['2012', '2013', '2014']
    for d in dirs:    
        parts = d.split('_')        
        basis, ep = str(parts[0]).split("ep") if "ep" in str(parts[0]) else (parts[0], '')
        basis, ep = basis.strip("-_"), ep.strip("-_")
        no_idx = parts.index('no') if 'no' in parts else -1
        upd_idx = parts.index('upd') if 'upd' in parts else -1
        upd = ''
        if no_idx != -1: upd += 'no_'
        if upd_idx != -1: upd += 'upd'    
        score = '_'.join(parts[1:] if upd == '' else parts[upd_idx+1:])

        for year in years:
            if new and year in ['2013', '2014']:
                with open(os.path.join(dirname, d, year+'_formal', 'testing_result'), 'r') as fin:
                    f1_score = float(fin.read().split('\t')[-1].strip().split(':')[-1].strip()) * 100
                    results.append(
                        {'Basis':basis, 'Epoch': ep, 'Update':upd, 'Score':score, 'Year':year, 'F1':f1_score}
                    )
            else:
                with open(os.path.join(dirname, d, year, 'F1'), 'r') as fin:
                    f1_score = float(fin.read().strip()) * 100
                    results.append(
                        {'Basis':basis, 'Epoch': ep, 'Update':upd, 'Score':score, 'Year':year, 'F1':f1_score}
                    )    
    
    results_index = pd.MultiIndex.from_tuples(
        [(result['Score'], result['Year'], result['Basis'], result['Epoch']) for result in results], \
        names=['Score', 'Year', 'Basis', 'Epoch']
    )
    results = pd.DataFrame([(result['Update'], result['F1']) for result in results], \
                           index=results_index, columns=['Update', 'F1'])
    
    if (results['Update'] == '').all(): results.drop(columns=['Update'], inplace=True)
    if (results.index.get_level_values(-1).values == '').all(): results = results.droplevel('Epoch')    
    return results.sort_index()

def gather_f1_es(dirname):
    dirs = [d for d in next(os.walk('./' + dirname))[1] if not d.startswith('.')]
    # print(dirs)
    results = []
    test_sets = ['es-train', 'es-test_spa_2012', 'es-all', 'es_2016_pilot_spa', 'es_2016_spa']
    is_dev_set = [True, False, True, False, False]
    
    for d in dirs:    
        parts = d.split('_')        
        basis, ep = str(parts[0]).split("ep") if "ep" in str(parts[0]) else (parts[0], '')
        basis, ep = basis.strip("-_"), ep.strip("-_")          
        score = '_'.join(parts[1:])

        for i, test_set in enumerate(test_sets):
            if not is_dev_set[i]:
                with open(os.path.join(dirname, d, test_set, 'testing_result'), 'r') as fin:
                    f1_score = float(fin.read().split('\t')[-1].strip().split(':')[-1].strip()) * 100
                    results.append(
                        {'Basis':basis, 'Epoch': ep, 'Score':score, 'TestSet':test_set, 'F1':f1_score}
                    )
            else:
                with open(os.path.join(dirname, d, test_set, 'F1'), 'r') as fin:
                    f1_score = float(fin.read().strip()) * 100
                    results.append(
                        {'Basis':basis, 'Epoch': ep, 'Score':score, 'TestSet':test_set, 'F1':f1_score}
                    )    
    
    results_index = pd.MultiIndex.from_tuples(
        [(result['Score'], result['TestSet'], result['Basis'], result['Epoch']) for result in results], \
        names=['Score', 'TestSet', 'Basis', 'Epoch']
    )
    results = pd.DataFrame([result['F1'] for result in results], \
                           index=results_index, columns=['F1'])
    if (results.index.get_level_values(-1).values == '').all(): results = results.droplevel('Epoch')        
    return results.sort_index()

In [3]:
def display_results(dirname, new=False):    
    results = gather_f1(dirname, new)    
    
    kmeans_df = results.loc['kmeans_avg'].rename(columns={'F1': 'KMeans Avg. F1'})    
    cos_df = results.loc['cos'].rename(columns={'F1': 'Cosine F1'})
    sc_df = results.loc['SC_avg'].rename(columns={'F1': 'SC Avg. F1'})
    
    display(pd.concat([kmeans_df, cos_df, sc_df], axis=1).sort_values(['Year', 'Basis']))
    return results

def display_results_es(dirname):    
    results = gather_f1_es(dirname)    
    
    kmeans_df = results.loc['kmeans_avg'].rename(columns={'F1': 'KMeans Avg. F1'})    
    cos_df = results.loc['cos'].rename(columns={'F1': 'Cosine F1'})
    sc_df = results.loc['SC_avg'].rename(columns={'F1': 'SC Avg. F1'})
    
    display(pd.concat([kmeans_df, cos_df, sc_df], axis=1).sort_values(['TestSet', 'Basis']))
    return results

In [4]:
results = {}

# <font color='green'>Milestone Transformer Run</font>

In [5]:
results["milestone_run_trans-b5-kb11"] = display_results("milestone_run_trans-b5-kb11_trans_results", True)

KMeans Avg. F1  Cosine F1  SC Avg. F1
Year Basis Epoch                                       
2012 b5    15          30.214286  19.824272   29.981855
           20          30.112195  19.632882   29.857300
           25          30.127040  19.483203   29.963368
           30          30.242828  19.431846   29.937193
           50          30.353883  19.572495   29.801086
2013 b5    15          35.808194  24.712992   35.183734
           20          37.269940  23.995405   37.279016
           25          36.852205  23.690078   36.505460
           30          38.505968  23.877442   37.597916
           50          37.322044  22.996709   37.751618
2014 b5    15          31.596267  19.639279   31.021300
           20          32.863340  18.675353   33.224398
           25          31.704670  17.776203   30.906060
           30          32.547945  17.535704   32.200733
           50          31.250000  16.666666   31.830240